<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re

import spacy
nlp = spacy.load('en_core_web_lg')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

In [2]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [5]:
yelp.shape

(10000, 9)

In [6]:
yelp.isnull().sum()

business_id    0
cool           0
date           0
funny          0
review_id      0
stars          0
text           0
useful         0
user_id        0
dtype: int64

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [7]:
def tokenize(doc):
    return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [8]:
yelp['token'] = yelp['text'].apply(tokenize)

In [10]:
yelp['token'][4]

['stars',
 'bad',
 'price',
 'lunch',
 'seniors',
 'pay',
 'eating',
 'hot',
 'food',
 'salads',
 'noodle',
 'bar',
 'dessert',
 'fruits',
 'fried',
 'varieties',
 'soup',
 'seating',
 'middle',
 'section',
 'bit',
 'tight',
 'booths',
 'look',
 'spacious',
 'think',
 'lunch',
 'noodle',
 'bar',
 'variety',
 'different',
 'types',
 'greens',
 'vermicelli',
 'noodles',
 'frozen',
 'tofu',
 'tomato',
 'laksa',
 'broth',
 'pretty',
 'good']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [12]:
# Instantiate vector object
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', min_df=0.025, max_df=0.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
sparse = tfidf.fit_transform(yelp['text'])

# Convert sparse matrix to dense matrix and get column names
matrix = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

matrix.head()

,able,absolutely,actually,ago,amazing,area,arrived,ask,asked,atmosphere,...,wonderful,work,working,worst,worth,wouldn,wrong,year,years,yelp
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.126168,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.149281,0.0,0.0
3,0.0,0.0,0.302932,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [13]:
# Calculate distance of TF-IDF vectors using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix = cosine_similarity(matrix)

In [14]:
dist = pd.DataFrame(dist_matrix)
dist.shape

(10000, 10000)

In [15]:
# Find top-10 most similar reviews (getting 11 since the first one will be the original document)
dist[0].nlargest(11)

0       1.000000
6899    0.528259
3714    0.524828
5465    0.470613
2131    0.470215
2865    0.460342
9036    0.438254
8470    0.430154
8531    0.427323
8411    0.413249
587     0.412669
Name: 0, dtype: float64

In [16]:
# Fit knn on TF-IDF vectors
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(matrix)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [20]:
custom_review= ["The foods were delicious, customer service was top-notch, price was reasonable. I will surely be this restaurant's loyal customer from now on"]

In [21]:
# Transform new review
new_vect = tfidf.transform(custom_review)

# Run model on new review
nn.kneighbors(new_vect.todense())

(array([[0.89722201, 0.89832692, 0.91611607, 0.94546797, 0.94713815,
         0.95891144, 0.96219901, 0.9624803 , 0.97061356, 0.97205653]]),
 array([[ 974, 6169, 3116, 3746, 3507, 2086, 4352, 6790, 8415, 5012]],
       dtype=int64))

In [24]:
index = [974, 6169, 3116, 3746, 3507, 2086, 4352, 6790, 8415, 5012]
top_ten = custom_review

for index in indicies:
    review = yelp['text'][index]
    print(review, '\n')

Britney at the front desk helped me out in an emergency. Her customer service was excellent I would hire her myself if she lived in my city 

great customer service and a amazing collection of reptiles and supplies. 

Airlines can not control the weather, but they can control customer service!  
I had four flights with Air Canada for my 15 year anniversary vacation.  We planned over two years and had a wonderful time.  The travel to and from was not great.  In fact, it almost ruined it for us.  
One weather delay turned into a mechanical delay which meant we were missing our connection to Europe via Toronto.  The stress would have been much less if Air Canada had proper staffing level to handle the overflowing queues.  I suggest a triage person to help people quickly!
Other airlines that had the same issues for me in the past gave great customer service which gave me comfort.  Some of the customer service reps were condescending and felt like it was my fault that I was in this situatio

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [26]:
# Train, test set
X = yelp['text']
y = yelp['stars']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Create pipeline components
vect = TfidfVectorizer(tokenizer=tokenize, stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

# Define the pipeline
pipe = Pipeline([('vect', vect), ('clf', rfc)])

# Define grid-search parameters
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500, 1000),
    'clf__n_estimators': (25, 100),
    'clf__max_depth': (15, 20)
}


In [35]:
grid_search = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   49.2s finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [36]:
grid_search.best_score_

0.5523750000000001

In [38]:
grid_search.best_estimator_.predict(custom_review)

array([5], dtype=int64)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [39]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [41]:
id2word = corpora.Dictionary(yelp['token'])

Create a bag of words representation of the entire corpus

In [42]:
corpus = [id2word.doc2bow(text) for text in yelp['token']]

Your LDA model should be ready for estimation: 

In [43]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [44]:
pyLDAvis.enable_notebook()

In [45]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.006275 -0.000324       1        1  10.601106
9     -0.001020 -0.003123       2        1  10.461185
2      0.000209  0.004585       3        1  10.353347
8     -0.000446 -0.000315       4        1  10.272483
4      0.001955 -0.000205       5        1  10.060967
0     -0.001835  0.001775       6        1   9.966220
5      0.000164 -0.000694       7        1   9.803439
3      0.002595 -0.001977       8        1   9.593763
1      0.002627  0.003363       9        1   9.561525
6      0.002027 -0.003085      10        1   9.325962, topic_info=           Term         Freq        Total Category  logprob  loglift
208       great  4127.000000  4127.000000  Default  30.0000  30.0000
133       place  4560.000000  4560.000000  Default  29.0000  29.0000
40         good  4458.000000  4458.000000  Default  28.0000  28.0000
440         day  1165.000000  1165.000000  Default  27.0000  27.0000
144        food  4752.000000  4752.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
735        love   128.951706  1588.222778  Topic10  -5.8508  -0.1386
310         try   116.990410  1401.014526  Topic10  -5.9481  -0.1105
358  definitely   116.410484  1395.139038  Topic10  -5.9531  -0.1113
115          ve   116.323112  1856.683594  Topic10  -5.9539  -0.3978
192     amazing   111.557312  1311.841064  Topic10  -5.9957  -0.0923

[870 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
1679       1  0.093322        able
1679       2  0.124430        able
1679       3  0.103691        able
1679       4  0.082953        able
1679       5  0.088138        able
...      ...       ...         ...
13097      8  0.065333         yuk
13097      9  0.196000         yuk
24360      2  0.323277     yummers
21303      9  0.617764  とても気に入りました
21304      9  0.617770    探したお店でした

[2919 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 3, 9, 5, 1, 6, 4, 2, 7])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)